# Loop Workflow Example 3

* High level approach to making a 3D model from just a bounding box and source files as input. (In Australia only for now. Documentation to come)
* This notebook uses example data provided by the Geological Survey of Western Australia.
* To run with your own data you will need to run the Utility 1 notebook to create an hjson config file and supply your own dtm model (service or geotif file in local coordinate system)

In [ ]:
#if not already installed:
#!conda install -c loop3d map2loop loopstructural pyamg meshio
#!pip install loopstructuralvisualisation[jupyter]

# note on versions:

## map2loop version = v3.2.0
## LoopStructural version = 1.6.4
## LoopProjectFile = 0.2.2
## loopstructural-visualisation = 0.1.10

## Map2Loop

In [ ]:
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.m2l_enums import Datatype
from map2loop.sampler import SamplerSpacing, SamplerDecimator
from map2loop.sorter import SorterUseHint, SorterUseNetworkX, SorterAgeBased, SorterAlpha

import os

# Specify the boundary of the region of interest in the appropriate projection coordinates
bounding_box = {
    "minx": 520000,
    "miny": 7490000,
    "maxx": 550000,
    "maxy": 7510000,
    "base": -3200,
    "top": 1200,
}

# define the name of the LoopProjectFile to be created
loop_project_filename = os.path.join( "local_source.loop3d")

# define the configuration dictionary to be used by map2loop (this replaces the legacy .hjson file)
config_dictionary = {
  "structure" : {
    "dipdir_column": "azimuth2",
    "dip_column": "dip",},
  "geology" : {
    "unitname_column": "unitname",
    "alt_unitname_column": "code"}, 
  "fault": {
      "minimum_fault_length": 0}
}


# Initialise the project with the shapefiles, dtm, config file
# output locations and projection to work in
proj = Project( 
    geology_filename = "./source_data/geol_clip_no_gaps.shp", # required
    fault_filename = "./source_data/faults_clip.shp", # optional
    structure_filename = "./source_data/structure_clip.shp", #required
    dtm_filename = './source_data/dtm_rp.tif', # required, 
    config_dictionary = config_dictionary,
    clut_filename = './source_data/500kibg_colours.csv', # optional: if this file is not provided, random colours will be used in map2loop
    verbose_level = VerboseLevel.NONE,
    working_projection = "EPSG:28350",
    bounding_box = bounding_box,
    loop_project_filename = loop_project_filename,
    overwrite_loopprojectfile = True, 
)

# Remove faults less than 5km
proj.set_minimum_fault_length(5000.0) # this updates the value set on the config_dictionary

# Set sampling distance for geology and fault maps to 200m
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(200.0))
proj.set_sampler(Datatype.FAULT, SamplerSpacing(200.0))

# Set to only take every second orientation observation (0 or 1 means take all observations)
proj.set_sampler(Datatype.STRUCTURE, SamplerDecimator(2))

# Set what text is expected for intrusions (contained within the description field)
proj.map_data.config.geology_config["intrusive_text"] = "mafic intrusive"

# Set specific layers from the geology map to be ignored (commonly "cover" or "water")
proj.set_ignore_lithology_codes(["cover", "Fortescue_Group", "A_FO_od"])

# Set a fault to be ignored (by name)
# proj.set_ignore_codes(["Fault_9"])

# Specify which stratigraphic columns sorter to use, other options are
# (SorterAlpha, SorterAgeBased, SorterUseHint, SorterUseNetworkX, SorterMaximiseContacts, SorterObservationProjections)
# proj.set_sorter(SorterAlpha())

# Or you can run map2loop and pre-specify the stratigraphic column
column = [
    # youngest
    'Turee_Creek_Group',
    'Boolgeeda_Iron_Formation',
    'Woongarra_Rhyolite',
    'Weeli_Wolli_Formation',
    'Brockman_Iron_Formation',
    'Mount_McRae_Shale_and_Mount_Sylvia_Formation',
    'Wittenoom_Formation',
    'Marra_Mamba_Iron_Formation',
    'Jeerinah_Formation',
    'Bunjinah_Formation',
    'Pyradie_Formation',
    'Fortescue_Group',
    # oldest
]
proj.run_all(user_defined_stratigraphic_column=column)

# Or you can get map2loop to run all column sorting algorithms it has and takes the one
# that has the longest total basal contact length
# proj.run_all(take_best=True)

In [ ]:
# Draw overlay of point data on geology map
# (options are 'basal_contacts', contacts','orientations','faults')
proj.draw_geology_map(overlay="orientations")

In [ ]:
# Extract estimate of the stratigraphic column
proj.stratigraphic_column.column

In [ ]:
proj.stratigraphic_column.stratigraphicUnits

## Loop Structural

In [ ]:
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor
from LoopStructural.visualisation import Loop3DView
import numpy as np

# LoopStructural Modelling
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e4,  # how many tetras/voxels
    'regularisation':5,
}
projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)

In [ ]:
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
model.nsteps=np.array([200,200,50])
model.update()

In [ ]:
view = Loop3DView(model)
view.plot_model_surfaces()
view.display()